In [ ]:
!pip install Flask

from collections import defaultdict
from flask import Flask, request, jsonify, Response
import pandas as pd
import numpy as np
import pickle
import json
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install pyngrok
!ngrok authtoken 2Xiq3njrFD1bBTy6pCGqHjSxUaU_6V8fJioQysEkv6jmc7q9N
from pyngrok import ngrok
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


In [ ]:

app = Flask(__name__)
run_with_ngrok(app)


In [ ]:
!pip install transformers
import pickle
from scipy.special import softmax
def preprocess(msg):
  X = []
  stop_words = set(nltk.corpus.stopwords.words("english"))
  tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
  tmp = []
  sentences = nltk.sent_tokenize(msg)
  for sent in sentences:
      sent = sent.lower()
      tokens = tokenizer.tokenize(sent)
      filtered_words = [w.strip() for w in tokens if w not in stop_words and len(w) > 1]
      tmp.extend(filtered_words)
  X.append(tmp)

  with open('/content/drive/MyDrive/Ml Pickle/tokenizer.pkl', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)
  X = tokenizer.texts_to_sequences(X)

  maxlen = 700
  X = pad_sequences(X, maxlen=maxlen)
  return X

def sent_result(msg):
  senti_model = "/content/drive/MyDrive/Ml Pickle/sentiment_model.pkl"
  senti_token = "/content/drive/MyDrive/Ml Pickle/sentiment_tokenizer.pkl"

  with open(senti_model, 'rb') as model_file:
    loaded_model = pickle.load(model_file)
  with open(senti_token, 'rb') as model_file:
    tokenizer = pickle.load(model_file)

  encoded_text = tokenizer(msg, return_tensors = 'pt')
  output = loaded_model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  val = 0
  acc = 0
  for i in range(3):
    if scores[i] > acc:
      acc = scores[i]
      val = i
  acc *= 100
  return [val, acc]

def initiate(msg):
  model_file_path = "/content/drive/MyDrive/Ml Pickle/model_pickle.pkl"
  with open(model_file_path, 'rb') as model_file:
    loaded_model = pickle.load(model_file)
  result = loaded_model.predict(preprocess(msg))
  fin = float(result)

  temp = sent_result(msg)
  ans = [fin, temp]
  return ans;

In [ ]:
@app.route('/upload', methods=['POST'])
def start():
  msg = request.json['sentence']
  result = initiate(msg)

  return jsonify(result)

In [ ]:
if __name__ == '__main__':
    app.run()